#  Ridge Regression (interpretation)

In this notebook, we will run ridge regression multiple times with different L2 penalties to see which one produces the best fit. We will revisit the example of polynomial regression as a means to see the effect of L2 regularization. In particular, we will:
* Use a pre-built implementation of regression (Turi Create) to run polynomial regression
* Use matplotlib to visualize polynomial regressions
* Use a pre-built implementation of regression (Turi Create) to run polynomial regression, this time with L2 penalty
* Use matplotlib to visualize polynomial regressions under L2 regularization
* Choose best L2 penalty using cross-validation.
* Assess the final fit using test data.

We will continue to use the House data from previous notebooks.  (In the next programming assignment for this module, you will implement your own ridge regression learning algorithm using gradient descent.)

# Fire up Turi Create

In [1080]:
import turicreate

# Polynomial regression, revisited

We build on the material from Week 3, where we wrote the function to produce an SFrame with columns containing the powers of a given input. Copy and paste the function `polynomial_sframe` from previous assignment:

In [1081]:
def polynomial_features(data, deg):
    data_copy = data.copy()
    for i in range(1, deg):
        data_copy['X' + str(i + 1)] = data_copy['X' + str(i)] * data_copy['X1']
    return data_copy

Let's use matplotlib to visualize what a polynomial regression looks like on the house data.

In [1082]:
import matplotlib.pyplot as plt
%matplotlib inline

In [1083]:
sales = turicreate.SFrame('home_data.sframe/')

As in Week 3, we will use the sqft_living variable. For plotting purposes (connecting the dots), you'll need to sort by the values of sqft_living. For houses with identical square footage, we break the tie by their prices.

In [1084]:
sales = sales.sort(['sqft_living','price'])

Let us revisit the 15th polynomial model using the 'sqft_living' input.Generate polynomial features up to degree 15 using `polynomial_features()`and fit a model with these features. When fitting the model,use an L2 penalty of `1e-5`:


In [1085]:
l2_small_penalty = 1e-5
x = sales['sqft_living']
y = sales['price']
data_frame = turicreate.SFrame({'X1': x})
data_frame = data_frame.add_column(y, column_name='price')

Note: When we have so many features and so few data points, the solution can become highly numerically unstable, which can sometimes lead to strange unpredictable results.  Thus, rather than using no regularization, we will introduce a tiny amount of regularization (`l2_penalty=1e-5`) to make the solution numerically stable.  (In lecture, we discussed the fact that regularization can also help with numerical stability, and here we are seeing a practical example.)

With the L2 penalty specified above, fit the model and print out the learned weights.

Hint: make sure to add 'price' column to the new SFrame before calling `turicreate.linear_regression.create()`. Also, make sure Turi Create doesn't create its own validation set by using the option `validation_set=None` in this call.

In [1086]:
def polynomial_regression(data, deg, l2_penalty, validation_set=None):
    model = turicreate.linear_regression.create(polynomial_features(data, deg),
                                                target='price',
                                                l2_penalty=l2_penalty,
                                                l1_penalty=0.,
                                                validation_set=validation_set,
                                                verbose=False)    
    
    return model

model = polynomial_regression(data=data_frame, deg=15, l2_penalty=l2_small_penalty)
print(model)

Class                          : LinearRegression

Schema
------
Number of coefficients         : 16
Number of examples             : 21613
Number of feature columns      : 15
Number of unpacked features    : 15

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.0

Training Summary
----------------
Solver                         : newton
Solver iterations              : 1
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.0344

Settings
--------
Residual sum of squares        : 1304281816164108.0
Training RMSE                  : 245656.4622

Highest Positive Coefficients
-----------------------------
(intercept)                    : 167924.8706
X1                             : 103.0909
X2                             : 0.1346
X4                             : 0.0
X6                             : 0.0

Lowest Negative Coefficients
----------------------------
X3                          

***QUIZ QUESTION:  What's the learned value for the coefficient of feature `power_1`?***

### Learned rate of coeficient 1

X1 : 103.0909

# Observe overfitting

Recall from Week 3 that the polynomial fit of degree 15 changed wildly whenever the data changed. In particular, when we split the sales data into four subsets and fit the model of degree 15, the result came out to be very different for each subset. The model had a *high variance*. We will see in a moment that ridge regression reduces such variance. But first, we must reproduce the experiment we did in Week 3.

First, split the data into split the sales data into four subsets of roughly equal size and call them `set_1`, `set_2`, `set_3`, and `set_4`. Use `.random_split` function and make sure you set `seed=0`. 

In [1087]:
(semi_split1, semi_split2) = sales.random_split(.5,seed=0)
(set_1, set_2) = semi_split1.random_split(0.5, seed=0)
(set_3, set_4) = semi_split2.random_split(0.5, seed=0)

Next, fit a 15th degree polynomial on `set_1`, `set_2`, `set_3`, and `set_4`, using 'sqft_living' to predict prices. Print the weights and make a plot of the resulting model.

Hint: When calling `turicreate.linear_regression.create()`, use the same L2 penalty as before (i.e. `l2_small_penalty`).  Also, make sure Turi Create doesn't create its own validation set by using the option `validation_set = None` in this call.

In [1088]:
x = set_1['sqft_living']
y = set_1['price']
data_frame = turicreate.SFrame({'X1': x})
data_frame = data_frame.add_column(y, column_name='price')

set_1_model = polynomial_regression(data_frame, 15, l2_penalty=l2_small_penalty)
print(set_1_model)

Class                          : LinearRegression

Schema
------
Number of coefficients         : 16
Number of examples             : 5404
Number of feature columns      : 15
Number of unpacked features    : 15

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.0

Training Summary
----------------
Solver                         : newton
Solver iterations              : 1
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.0046

Settings
--------
Residual sum of squares        : 334244159982603.5
Training RMSE                  : 248699.1173

Highest Positive Coefficients
-----------------------------
(intercept)                    : 9306.4651
X1                             : 585.8658
X3                             : 0.0001
X6                             : 0.0
X7                             : 0.0

Lowest Negative Coefficients
----------------------------
X2                             :

In [1089]:
x = set_2['sqft_living']
y = set_2['price']
data_frame = turicreate.SFrame({'X1': x})
data_frame = data_frame.add_column(y, column_name='price')

set_2_model = polynomial_regression(data_frame, 15, l2_penalty=l2_small_penalty)
print(set_2_model)

Class                          : LinearRegression

Schema
------
Number of coefficients         : 16
Number of examples             : 5398
Number of feature columns      : 15
Number of unpacked features    : 15

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.0

Training Summary
----------------
Solver                         : newton
Solver iterations              : 1
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.0047

Settings
--------
Residual sum of squares        : 296922466397560.1
Training RMSE                  : 234533.6106

Highest Positive Coefficients
-----------------------------
X1                             : 783.4938
X3                             : 0.0004
X5                             : 0.0
X6                             : 0.0
X10                            : 0.0

Lowest Negative Coefficients
----------------------------
(intercept)                    : -2511

In [1090]:
x = set_3['sqft_living']
y = set_3['price']
data_frame = turicreate.SFrame({'X1': x})
data_frame = data_frame.add_column(y, column_name='price')

set_3_model = polynomial_regression(data_frame, 15, l2_penalty=l2_small_penalty)
print(set_3_model)

Class                          : LinearRegression

Schema
------
Number of coefficients         : 16
Number of examples             : 5409
Number of feature columns      : 15
Number of unpacked features    : 15

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.0

Training Summary
----------------
Solver                         : newton
Solver iterations              : 1
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.0093

Settings
--------
Residual sum of squares        : 341037823457605.44
Training RMSE                  : 251097.7281

Highest Positive Coefficients
-----------------------------
(intercept)                    : 462426.5778
X2                             : 1.0287
X4                             : 0.0
X7                             : 0.0
X8                             : 0.0

Lowest Negative Coefficients
----------------------------
X1                             : -

In [1091]:
x = set_4['sqft_living']
y = set_4['price']
data_frame = turicreate.SFrame({'X1': x})
data_frame = data_frame.add_column(y, column_name='price')

set_4_model = polynomial_regression(data_frame, 15, l2_penalty=l2_small_penalty)
print(set_4_model)

Class                          : LinearRegression

Schema
------
Number of coefficients         : 16
Number of examples             : 5402
Number of feature columns      : 15
Number of unpacked features    : 15

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.0

Training Summary
----------------
Solver                         : newton
Solver iterations              : 1
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.0078

Settings
--------
Residual sum of squares        : 322513810180898.75
Training RMSE                  : 244341.2932

Highest Positive Coefficients
-----------------------------
X1                             : 1247.5904
X3                             : 0.0006
X6                             : 0.0
X7                             : 0.0
X10                            : 0.0

Lowest Negative Coefficients
----------------------------
(intercept)                    : -17

The four curves should differ from one another a lot, as should the coefficients you learned.

***QUIZ QUESTION:  For the models learned in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?***  (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

### Smallest Value

X1: -759.2519 from the model created on the 3rd set.

### Largest Value
X1 : 1247.5904 from the model created on the 4th set.



Generally, whenever we see weights change so much in response to change in data, we believe the variance of our estimate to be large. Ridge regression aims to address this issue by penalizing "large" weights. (Weights of `model15` looked quite small, but they are not that small because 'sqft_living' input is in the order of thousands.)

With the argument `l2_penalty=1e5`, fit a 15th-order polynomial model on `set_1`, `set_2`, `set_3`, and `set_4`. Other than the change in the `l2_penalty` parameter, the code should be the same as the experiment above. Also, make sure Turi Create doesn't create its own validation set by using the option `validation_set = None` in this call.

# Ridge regression comes to rescue

In [1092]:

data_frame = turicreate.SFrame({'X1': x})
data_frame = data_frame.add_column(y, column_name='price')
set_1_model = polynomial_regression(data_frame, 15, l2_penalty=1e5)
print(set_1_model)

Class                          : LinearRegression

Schema
------
Number of coefficients         : 16
Number of examples             : 5402
Number of feature columns      : 15
Number of unpacked features    : 15

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 100000.0

Training Summary
----------------
Solver                         : newton
Solver iterations              : 1
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.0082

Settings
--------
Residual sum of squares        : 563974715173920.4
Training RMSE                  : 323111.5829

Highest Positive Coefficients
-----------------------------
(intercept)                    : 513667.0871
X1                             : 1.9104
X2                             : 0.0011
X3                             : 0.0
X4                             : 0.0

Lowest Negative Coefficients
----------------------------
No Negative Coefficients   

In [1093]:
x = set_2['sqft_living']
y = set_2['price']
data_frame = turicreate.SFrame({'X1': x})
data_frame = data_frame.add_column(y, column_name='price')

set_2_model = polynomial_regression(data_frame, 15, l2_penalty=1e5)
print(set_2_model)

Class                          : LinearRegression

Schema
------
Number of coefficients         : 16
Number of examples             : 5398
Number of feature columns      : 15
Number of unpacked features    : 15

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 100000.0

Training Summary
----------------
Solver                         : newton
Solver iterations              : 1
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.0047

Settings
--------
Residual sum of squares        : 564001004832112.1
Training RMSE                  : 323238.8096

Highest Positive Coefficients
-----------------------------
(intercept)                    : 519216.8974
X1                             : 2.0447
X2                             : 0.0011
X3                             : 0.0
X4                             : 0.0

Lowest Negative Coefficients
----------------------------
No Negative Coefficients   

In [1094]:
x = set_3['sqft_living']
y = set_3['price']
data_frame = turicreate.SFrame({'X1': x})
data_frame = data_frame.add_column(y, column_name='price')

set_3_model = polynomial_regression(data_frame, 15, l2_penalty=1e5)
print(set_3_model)

Class                          : LinearRegression

Schema
------
Number of coefficients         : 16
Number of examples             : 5409
Number of feature columns      : 15
Number of unpacked features    : 15

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 100000.0

Training Summary
----------------
Solver                         : newton
Solver iterations              : 1
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.0084

Settings
--------
Residual sum of squares        : 662729427751762.8
Training RMSE                  : 350033.5213

Highest Positive Coefficients
-----------------------------
(intercept)                    : 522911.518
X1                             : 2.2689
X2                             : 0.0013
X3                             : 0.0
X4                             : 0.0

Lowest Negative Coefficients
----------------------------
No Negative Coefficients    

In [1095]:
x = set_4['sqft_living']
y = set_4['price']
data_frame = turicreate.SFrame({'X1': x})
data_frame = data_frame.add_column(y, column_name='price')

set_4_model = polynomial_regression(data_frame, 15, l2_penalty=1e5)
print(set_4_model)

Class                          : LinearRegression

Schema
------
Number of coefficients         : 16
Number of examples             : 5402
Number of feature columns      : 15
Number of unpacked features    : 15

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 100000.0

Training Summary
----------------
Solver                         : newton
Solver iterations              : 1
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.0077

Settings
--------
Residual sum of squares        : 563974715173920.4
Training RMSE                  : 323111.5829

Highest Positive Coefficients
-----------------------------
(intercept)                    : 513667.0871
X1                             : 1.9104
X2                             : 0.0011
X3                             : 0.0
X4                             : 0.0

Lowest Negative Coefficients
----------------------------
No Negative Coefficients   

These curves should vary a lot less, now that you applied a high degree of regularization.

***QUIZ QUESTION:  For the models learned with the high level of regularization in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?*** (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

### Largest

The largest was X1: 2.26  in set 1.

### Smallest

The smallest was X1:1.91 in set 3.

# Selecting an L2 penalty via cross-validation

Just like the polynomial degree, the L2 penalty is a "magic" parameter we need to select. We could use the validation set approach as we did in the last module, but that approach has a major disadvantage: it leaves fewer observations available for training. **Cross-validation** seeks to overcome this issue by using all of the training set in a smart way.

We will implement a kind of cross-validation called **k-fold cross-validation**. The method gets its name because it involves dividing the training set into k segments of roughtly equal size. Similar to the validation set method, we measure the validation error with one of the segments designated as the validation set. The major difference is that we repeat the process k times as follows:

Set aside segment 0 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
Set aside segment 1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
...<br>
Set aside segment k-1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set

After this process, we compute the average of the k validation errors, and use it as an estimate of the generalization error. Notice that  all observations are used for both training and validation, as we iterate over segments of data. 

To estimate the generalization error well, it is crucial to shuffle the training data before dividing them into segments. The package turicreate_cross_validation (see below) has a utility function for shuffling a given SFrame. We reserve 10% of the data as the test set and shuffle the remainder. (Make sure to use `seed=1` to get consistent answer.)

  
_Note:_ For applying cross-validation, we will import a package called `turicreate_cross_validation`. To install it, please run this command on your terminal:

`pip install -e git+https://github.com/Kagandi/turicreate-cross-validation.git#egg=turicreate_cross_validation`

You can find the documentation on this package here: https://github.com/Kagandi/turicreate-cross-validation

In [1096]:
import turicreate_cross_validation.cross_validation as tcv

(train_valid, test) = sales.random_split(.9, seed=1)
train_valid_shuffled = tcv.shuffle_sframe(train_valid, random_seed=1)

Once the data is shuffled, we divide it into equal segments. Each segment should receive `n/k` elements, where `n` is the number of observations in the training set and `k` is the number of segments. Since the segment 0 starts at index 0 and contains `n/k` elements, it ends at index `(n/k)-1`. The segment 1 starts where the segment 0 left off, at index `(n/k)`. With `n/k` elements, the segment 1 ends at index `(n*2/k)-1`. Continuing in this fashion, we deduce that the segment `i` starts at index `(n*i/k)` and ends at `(n*(i+1)/k)-1`.

With this pattern in mind, we write a short loop that prints the starting and ending indices of each segment, just to make sure you are getting the splits right.

In [1097]:
n = len(train_valid_shuffled)
k = 10 # 10-fold cross-validation

for i in range(k):
    start = (n*i)/k
    end = (n*(i+1))/k-1
    print(i, (start, end))

0 (0.0, 1938.6)
1 (1939.6, 3878.2)
2 (3879.2, 5817.8)
3 (5818.8, 7757.4)
4 (7758.4, 9697.0)
5 (9698.0, 11636.6)
6 (11637.6, 13576.2)
7 (13577.2, 15515.8)
8 (15516.8, 17455.4)
9 (17456.4, 19395.0)


Let us familiarize ourselves with array slicing with SFrame. To extract a continuous slice from an SFrame, use colon in square brackets. For instance, the following cell extracts rows 0 to 9 of `train_valid_shuffled`. Notice that the first index (0) is included in the slice but the last index (10) is omitted.

In [1098]:
train_valid_shuffled[0:10] # rows 0 to 9

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
8645511350,2014-12-01 00:00:00+00:00,300000.0,3.0,1.75,1810.0,21138.0,1.0,0
7237501370,2014-07-17 00:00:00+00:00,1079000.0,4.0,3.25,4800.0,12727.0,2.0,0
7278700100,2015-01-21 00:00:00+00:00,625000.0,4.0,2.5,2740.0,9599.0,1.0,0
1421079007,2015-03-24 00:00:00+00:00,408506.0,3.0,2.75,2480.0,209199.0,1.5,0
4338800370,2014-11-17 00:00:00+00:00,220000.0,3.0,1.0,1000.0,6020.0,1.0,0
7511200020,2014-08-29 00:00:00+00:00,509900.0,3.0,1.75,1690.0,53578.0,1.0,0
3300701615,2014-09-30 00:00:00+00:00,655000.0,4.0,2.5,2630.0,4000.0,3.0,0
7011200260,2014-12-19 00:00:00+00:00,485000.0,4.0,2.0,1400.0,3600.0,1.0,0
3570000130,2014-06-11 00:00:00+00:00,580379.0,4.0,2.75,2240.0,27820.0,1.5,0
2796100640,2015-04-24 00:00:00+00:00,264900.0,4.0,2.5,2040.0,7000.0,1.0,0


Now let us extract individual segments with array slicing. Consider the scenario where we group the houses in the `train_valid_shuffled` dataframe into k=10 segments of roughly equal size, with starting and ending indices computed as above.
Extract the fourth segment (segment 3) and assign it to a variable called `validation4`.

In [1099]:
"""Return a k data set between 1 and k.
Used for splitting sets up for X-validation.
param: data_set -- Data set that is split into k sets.
param: k -- The amount of sets to split data_set into.
param: segment_number: The k data set that you want.
Returns a k data set"""
def extract_k_set(data_set, k, segment_number):
    i = segment_number - 1
    n = len(data_set)
    starting_index = n * i / k
    ending_index = n * (i + 1) / k - 1
    return data_set[starting_index: ending_index]

In [1100]:
validation4 = extract_k_set(train_valid_shuffled, k, 4)

To verify that we have the right elements extracted, run the following cell, which computes the average price of the fourth segment. When rounded to nearest whole number, the average should be $559,642.

In [1101]:
print(int(round(validation4['price'].mean(), 0)))

559642


After designating one of the k segments as the validation set, we train a model using the rest of the data. To choose the remainder, we slice (0:start) and (end+1:n) of the data and paste them together. SFrame has `append()` method that pastes together two disjoint sets of rows originating from a common dataset. For instance, the following cell pastes together the first and last two rows of the `train_valid_shuffled` dataframe.

In [1102]:
n = len(train_valid_shuffled)
first_two = train_valid_shuffled[0:2]
last_two = train_valid_shuffled[n-2:n]
print(first_two.append(last_two))

+------------+---------------------------+-----------+----------+-----------+
|     id     |            date           |   price   | bedrooms | bathrooms |
+------------+---------------------------+-----------+----------+-----------+
| 8645511350 | 2014-12-01 00:00:00+00:00 |  300000.0 |   3.0    |    1.75   |
| 7237501370 | 2014-07-17 00:00:00+00:00 | 1079000.0 |   4.0    |    3.25   |
| 4077800582 | 2014-09-12 00:00:00+00:00 |  522000.0 |   3.0    |    1.0    |
| 7853370620 | 2015-02-06 00:00:00+00:00 |  605000.0 |   5.0    |    4.0    |
+------------+---------------------------+-----------+----------+-----------+
+-------------+----------+--------+------------+------+-----------+-------+
| sqft_living | sqft_lot | floors | waterfront | view | condition | grade |
+-------------+----------+--------+------------+------+-----------+-------+
|    1810.0   | 21138.0  |  1.0   |     0      |  0   |     4     |  7.0  |
|    4800.0   | 12727.0  |  2.0   |     0      |  0   |     3     |  10.

Extract the remainder of the data after *excluding* fourth segment (segment 3) and assign the subset to `train4`.

In [1103]:
segment_number = 3
n = len(train_valid_shuffled)
starting_index = n * segment_number / k
ending_index = n * (segment_number + 1) / k - 1

print(f"N: {n}")
print(f"starting index: {starting_index}")
print(f"ending index: {ending_index}")

training_set = train_valid_shuffled[0: 5818]
training_set.append(train_valid_shuffled[7758: n])
print(round(training_set['price'].mean(), 0))

print(training_set)

N: 19396
starting index: 5818.8
ending index: 7757.4
533153.0
+------------+---------------------------+-----------+----------+-----------+
|     id     |            date           |   price   | bedrooms | bathrooms |
+------------+---------------------------+-----------+----------+-----------+
| 8645511350 | 2014-12-01 00:00:00+00:00 |  300000.0 |   3.0    |    1.75   |
| 7237501370 | 2014-07-17 00:00:00+00:00 | 1079000.0 |   4.0    |    3.25   |
| 7278700100 | 2015-01-21 00:00:00+00:00 |  625000.0 |   4.0    |    2.5    |
| 1421079007 | 2015-03-24 00:00:00+00:00 |  408506.0 |   3.0    |    2.75   |
| 4338800370 | 2014-11-17 00:00:00+00:00 |  220000.0 |   3.0    |    1.0    |
| 7511200020 | 2014-08-29 00:00:00+00:00 |  509900.0 |   3.0    |    1.75   |
| 3300701615 | 2014-09-30 00:00:00+00:00 |  655000.0 |   4.0    |    2.5    |
| 7011200260 | 2014-12-19 00:00:00+00:00 |  485000.0 |   4.0    |    2.0    |
| 3570000130 | 2014-06-11 00:00:00+00:00 |  580379.0 |   4.0    |    2.75   |
| 

To verify that we have the right elements extracted, run the following cell, which computes the average price of the data with fourth segment excluded. When rounded to nearest whole number, the average should be $536,865.

In [1104]:
print(int(round(validation4['price'].mean(), 0)))

print(int(round(training_set['price'].mean(), 0)))

559642
533153


Now we are ready to implement k-fold cross-validation. Write a function that computes k validation errors by designating each of the k segments as the validation set. It accepts as parameters (i) `k`, (ii) `l2_penalty`, (iii) dataframe, (iv) name of output column (e.g. `price`) and (v) list of feature names. The function returns the average validation error using k segments as validation sets.

* For each i in [0, 1, ..., k-1]:
  * Compute starting and ending indices of segment i and call 'start' and 'end'
  * Form validation set by taking a slice (start:end+1) from the data.
  * Form training set by appending slice (end+1:n) to the end of slice (0:start).
  * Train a linear model using training set just formed, with a given l2_penalty
  * Compute validation error using validation set just formed

In [1105]:
def k_fold_cross_validation(k, l2_penalty_list, data, output_name, features_list, deg):

    # Starting and ending indexes for validation set.
    def get_starting_ending_indicies(segment_number, n):
        starting_index = n * segment_number / k
        ending_index = n * (segment_number + 1) / k - 1
        return starting_index, ending_index

    # Everything that is not in our validation set.
    def get_training_set(segment_number, n):
        start, end = get_starting_ending_indicies(segment_number=segment_number, n=n)
        training_set = data[0: start]
        training_set.append(train_valid_shuffled[ending_index + 1: n])
        return training_set
    
    # Return mse a model with the specified params.
    def best_l2_from_linear_model(training_data, validation_set, l2_penalty_list,
                                  output_name, features_list):

        # set x frame to have the proper features_list data.
        x = training_data[features_list[0]]
        for i in range(1, len(features_list)):
            x = training_data[features_list[i]]

        # This is the target that we are going to try to predict.
        y = training_data[output_name]
        
        # Now we make the data set that we can pass into our linear reg model.
        data_frame = turicreate.SFrame({'X1': x})
        data_frame = data_frame.add_column(y, column_name=output_name)

        # These fields are important for finding the smallest l2_penalty.
        smallest_mse = None
        best_l2_penalty = None
        
        # Alright, lets make each model on each l2_penalty given.
        for l2_penalty in l2_penalty_list:
            model = turicreate.linear_regression.create(polynomial_features(data_frame, deg),
                                                        target=output_name,
                                                        l2_penalty=l2_penalty,
                                                        l1_penalty=0.,
                                                        validation_set=validation_set,
                                                        verbose=False)
            predicted_value = model.predict(data)
            print(predicted_value)

            # We will take whatever the smallest mse is.

            # Update best_l2_penalty based on the mse value.

            exit()

    n = len(data)

    # The smallest mse means we found the best l2_penalty :)
    best_l2_penalty = 0.0

    for i in range(1, k - 1):
        # We need to find our indexes so we can get a k set from the current segment we are on.
        start, end = get_starting_ending_indicies(segment_number=i, n=n)

        # Let's split the data correctly into validation and training.
        validation_set = data[start: end]
        training_set = get_training_set(segment_number=i, n=n)
        
        # Now, let's find the best mse from the given l2 penalties.
        l2_penalty= get_mse_from_linear_model(training_data=training_set,
                                              validation_set=validation_set,
                                              l2_penalty_list=l2_penalty_list,
                                              output_name=output_name,
                                              features_list=features_list)


Once we have a function to compute the average validation error for a model, we can write a loop to find the model that minimizes the average validation error. Write a loop that does the following:
* We will again be aiming to fit a 15th-order polynomial model using the `sqft_living` input
* For `l2_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, you can use this Numpy function: `np.logspace(1, 7, num=13)`.)
    * Run 10-fold cross-validation with `l2_penalty`
* Report which L2 penalty produced the lowest average validation error.

Note: since the degree of the polynomial is now fixed to 15, to make things faster, you should generate polynomial features in advance and re-use them throughout the loop. Make sure to use `train_valid_shuffled` when generating polynomial features!

In [1108]:
import numpy as np

k_fold_cross_validation(k=10,
                        l2_penalty_list=np.logspace(1, 7, num=13),
                        data=train_valid_shuffled,
                        output_name='price',
                        features_list=['sqft_living'],
                        deg=15)

RuntimeError: Two SFrames have different number of columns

***QUIZ QUESTIONS:  What is the best value for the L2 penalty according to 10-fold validation?***

You may find it useful to plot the k-fold cross-validation errors you have obtained to better understand the behavior of the method.  

In [1107]:
# Plot the l2_penalty values in the x axis and the cross-validation error in the y axis.
# Using plt.xscale('log') will make your plot more intuitive.



Once you found the best value for the L2 penalty using cross-validation, it is important to retrain a final model on all of the training data using this value of `l2_penalty`. This way, your final model will be trained on the entire dataset.

***QUIZ QUESTION: Using the best L2 penalty found above, train a model using all training data. What is the RSS on the TEST data of the model you learn with this L2 penalty? ***